In [1]:
# define rooms and items

#game room

couch = {
    "name": "couch",
    "type": "furniture",
    "description": "It's just a couch. There isn't anything interesting about it."
}

door_a = {
    "name": "door a",
    "type": "door",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

piano = {
    "name": "piano",
    "type": "furniture",
    "description": "It's just a piano. There isn't anything interesting about it."
}

game_room = {
    "name": "game room",
    "type": "room",
}

chess_table = {
    "name": "chess table",
    "type": "furniture",
    "description": "It's just a chess table. There is no time to play."
}

#bedroom_1

bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}

door_b = {
    "name": "door b",
    "type": "door",
}    

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

queen_bed = {
    "name": "queen bed",
    "type": "furniture",
    "description": "It's just a large bed. There is no time for a nap."
}

lamp = {
    "name": "lamp",
    "type": "furniture",
    "description": "It's just a lamp. There isn't anything interesting about it."
}


door_c = {
    "name": "door c",
    "type": "door",
}

#bedroom_2

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

door_d = {
    "name": "door d",
    "type": "door",
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}

double_bed = {
    "name": "double bed",
    "type": "furniture",
    "description": "It's just a bed. There is no time for a nap."
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

portrait = {
    "name": "portrait",
    "type": "furniture",
    "description": "You found a piece of paper behind it that reads '385'."
}

chest = {
    "name": "chest",
    "type": "furniture",
    "description": "The chest is locked with a code."
}

#living_room

living_room = {
    "name": "living room",
    "type": "room",
}

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}

dining_table = {
    "name": "dining table",
    "type": "furniture",
    "description": "It's just a table. There isn't anything interesting about it."
}

door_e = {
    "name": "door e",
    "type": "trap",
}

#outside

outside = {
  "name": "outside"
}

all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a, chess_table],
    "piano": [key_a],
    "bedroom 1": [door_a, door_b, door_c, queen_bed, lamp],
    "door a": [game_room, bedroom_1],
    "queen bed": [key_b],
    "door b": [bedroom_1, bedroom_2],
    "bedroom 2": [double_bed, dresser, door_b, chest, portrait],
    "double bed": [key_c],
    "dresser": [key_d],
    "living room": [dining_table, door_d, door_c, door_e],
    "door c": [bedroom_1, living_room],
    "door d": [living_room, outside]
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": bedroom_1
}




In [2]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house. You have 12 turns before your host arrives, examine the items wisely!")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == outside):
        print("Congrats! You are free!")
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip().lower()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            # added list of items to examine by the input
            room_var = game_state["current_room"]
            item_found_lst = [item["name"] for item in object_relations[room_var["name"]]]
            examine_item(input("What would you like to examine? " + str(item_found_lst)).strip().lower())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name, turns=[0]):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ''
    output = None
    
    ## counts how many turns you have left
    turns[0] += 1
    turns_left = 12 - turns[0]
    if turns_left < 1:
        print("Your host has arrived! You're dead! Try again!")
        
    else:
        print("You have {} turns before your host arrive.".format(turns_left))
        for item in object_relations[current_room["name"]]:
            if(item["name"] == item_name):
                output = "You examine " + item_name + ". "
                
                ## trap door_e
                if(item["type"] == "trap"):
                    input_trap = input("The door is unlocked. Enter at your own risk. Enter 'yes' or 'no'").strip().lower()
                    if input_trap == 'yes':
                        print("It's a trap door! You're dead! Try again!")
                        return
                    
                elif(item["type"] == "door"):
                    have_key = False
                    for key in game_state["keys_collected"]:
                        if(key["target"] == item):
                            have_key = True
                    if(have_key):
                        output += "You unlock it with a key you have."
                        next_room = get_next_room_of_door(item, current_room)
                    else:
                        output += "It is locked but you don't have the key."
                        
                ##added the chest feature, if code is right, reset turns
                elif item["name"] == "chest":
                    output += item["description"]
                    code = input("Enter 3 digit code:").strip()
                    if code == '385':
                        print("You found a chocolate bar. Win 6 turns.")
                        turns[0] = 0
                        print("Your host will be late. You have 12 turns left.")
                    else:
                        print("That is not the code.")
                        
                else:
                    if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                        item_found = object_relations[item["name"]].pop()
                        game_state["keys_collected"].append(item_found)
                        output += "You find " + item_found["name"] + "."
                        
                        #added inventory
                        inventory = [items["name"] for items in game_state["keys_collected"]]
                        print("INVENTORY:", inventory)
                    else:
                        output += item["description"] #changed descriptions
                print(output)
                break

        if(output is None):
            print("The item you requested is not found in the current room.")

        if(next_room and input("Do you want to go to the next room? Enter 'yes' or 'no'").strip().lower() == 'yes'):
            game_state["target_room"] = next_room
            play_room(next_room)
        else:
            play_room(current_room)

In [3]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house. You have 12 turns before your host arrives, examine the items wisely!
You are now in game room
What would you like to do? Type 'explore' or 'examine'?examine
What would you like to examine? ['couch', 'piano', 'door a', 'chess table']piano
You have 11 turns before your host arrive.
INVENTORY: ['key for door a']
You examine piano. You find key for door a.
You are now in game room
What would you like to do? Type 'explore' or 'examine'?examine
What would you like to examine? ['couch', 'piano', 'door a', 'chess table']door a
You have 10 turns before your host arrive.
You examine door a. You unlock it with a key you have.
Do you want to go to the next room? Enter 'yes' or 'no'yes
You are now in bedroom 1
What would you like to do? Type 'e